## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-15-18-49-44 +0000,nyt,Afghanistan and Pakistan Announce Cease-Fire A...,https://www.nytimes.com/2025/10/15/world/asia/...
1,2025-10-15-18-49-21 +0000,nyt,Dallin H. Oaks Named President of the Mormon C...,https://www.nytimes.com/2025/10/14/us/dallin-o...
2,2025-10-15-18-49-19 +0000,nyt,Republicans Face Fallout From Racist and Homop...,https://www.nytimes.com/2025/10/15/us/politics...
3,2025-10-15-18-48-13 +0000,nypost,Disney World guest dies by apparent suicide on...,https://nypost.com/2025/10/15/us-news/disney-w...
4,2025-10-15-18-46-04 +0000,nyt,First Burials to Be Held for Israelis Returned...,https://www.nytimes.com/2025/10/15/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2473/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
78,trump,49
39,new,17
245,hamas,17
231,court,15
35,gaza,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
216,2025-10-15-09-30-00 +0000,wsj,Governors across 15 states are forming a new p...,https://www.wsj.com/health/healthcare/rfk-jr-h...,110
296,2025-10-14-23-18-03 +0000,wapo,U.S. kills 6 alleged drug smugglers in new str...,https://www.washingtonpost.com/national-securi...,94
192,2025-10-15-10-08-00 +0000,wsj,Israel and Hamas Start Next Phase of Talks on ...,https://www.wsj.com/world/middle-east/israel-a...,93
217,2025-10-15-09-24-36 +0000,bbc,Trump demands Hamas disarm as group moves to a...,https://www.bbc.com/news/articles/cjr034p5prlo...,88
130,2025-10-15-14-21-58 +0000,nyt,Trump Says He May Give Tomahawks to Ukraine. I...,https://www.nytimes.com/2025/10/14/us/politics...,88


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
216,110,2025-10-15-09-30-00 +0000,wsj,Governors across 15 states are forming a new p...,https://www.wsj.com/health/healthcare/rfk-jr-h...
208,64,2025-10-15-10-00-00 +0000,nypost,Dr. Keith Ablow tells ‘Pod Force One’ why he f...,https://nypost.com/2025/10/15/us-news/dr-keith...
324,62,2025-10-14-21-30-20 +0000,nyt,"Israel Pressures Hamas to Return Bodies, but G...",https://www.nytimes.com/2025/10/14/world/middl...
132,61,2025-10-15-14-11-10 +0000,latimes,Supreme Court might upend Voting Rights Act an...,https://www.latimes.com/politics/story/2025-10...
137,52,2025-10-15-13-59-09 +0000,bbc,Government will publish evidence in collapsed ...,https://www.bbc.com/news/articles/cql9v6x4wpzo...
360,46,2025-10-14-19-42-00 +0000,wsj,Maine’s Democratic Gov. Janet Mills officially...,https://www.wsj.com/politics/elections/susan-c...
123,38,2025-10-15-14-38-01 +0000,nypost,"Family of kidnapped IDF soldier, 18, only foun...",https://nypost.com/2025/10/15/world-news/idf-s...
118,36,2025-10-15-14-52-00 +0000,wsj,BlackRock’s AI Consortium to Buy Aligned Data ...,https://www.wsj.com/finance/investing/blackroc...
357,35,2025-10-14-19-55-46 +0000,bbc,Madagascar military says it has seized power a...,https://www.bbc.com/news/articles/cn8xjjdgl8vo...
302,34,2025-10-14-22-31-05 +0000,nypost,Trump honors late Charlie Kirk with Medal of F...,https://nypost.com/2025/10/14/us-news/trump-ho...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
